# Problem 2: Visualize an interactive map (10 points)

Create a nice interactive map using the skills you leared in lesson 5: interactive maps using Folium (or some other suitable package - feel free to experiment!). Write your code into a notebook file (.ipynb) or a python script file (.py) and store the output map(s) in .html format under the docs folder.

Topic of the map:

    You can select the topic of the map freely.
    This map should not be only an interactive version of your submission for problem 1 ;). Create something new!
    Feel free to adapt examples provided in this course!

Criteria:

    The map should have multiple layers on it and/or present an output of (simple) data analysis (something beyond plotting raw data on a map).
    Consider good cartographic practices
    The map should demonstrate skills learned during lesson 5: interactive maps and throughout this course. You can also take advantage of the Folium example gallery for further ideas.
    Think about including additional interactive elements, such as popup text in the map.

Output:

    Remember to commit the code and input data (or at least a link to input data)
    Save your map(s) in .html format in the docs folder


In [45]:
import folium
from pyproj import crs
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import glob
import numpy as np

In [46]:
# Read data

# Define the filepath for the grid

grid_fp = "data/MetropAccess_YKR_grid_EurefFIN.shp"

# Read the Polygon grid into a GeoDataFrame called "grid"

grid = gpd.read_file(grid_fp)

# Check the grid

grid.head()

,x,y,YKR_ID,geometry
0,381875.0,6697880.0,5785640,"POLYGON ((382000.000 6697750.000, 381750.000 6..."
1,382125.0,6697880.0,5785641,"POLYGON ((382250.000 6697750.000, 382000.000 6..."
2,382375.0,6697880.0,5785642,"POLYGON ((382500.000 6697750.000, 382250.000 6..."
3,382625.0,6697880.0,5785643,"POLYGON ((382750.000 6697750.000, 382500.000 6..."
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000 6697500.000, 381000.000 6..."


In [47]:
# List the file names into a list called "filepaths"

filepaths = glob.glob('data/TravelTimes_to_*_*.txt')
filepaths

# Check how many filepaths there are
print("Number of travel time data files: ", len(filepaths))

Number of travel time data files:  7


Now, let's join the data from all the text files into the grid. This time, instead of travel times by public transport, let's use travel times by car.

Required columns: `'YKR_ID', 'car_r_t_Jumbo', 'car_r_t_Dixi', 'car_r_t_Myyrmanni', 'car_r_t_Itis', 'car_r_t_Forum', 'car_r_t_IsoOmena', 'car_r_t_Ruoholahti', 'geometry'`

In [48]:
filelist = []

for path in filepaths:
    df = pd.read_csv(path, sep=";")
    df = df[['from_id', 'car_r_t']]
    df = df.rename(columns={"from_id": "YKR_ID"})
    p_text, p_ext = path.split('.')
    drop1, drop2, drop3, name = p_text.split('_')
    df = df.rename(columns={"car_r_t": "car_r_t_"+name})
    filelist.append(df)

In [49]:
filelist

[        YKR_ID  car_r_t_Jumbo
 0      5785640             39
 1      5785641             39
 2      5785642             45
 3      5785643             46
 4      5787544             38
 ...        ...            ...
 13226  6016698             58
 13227  6016699             60
 13228  6018252             -1
 13229  6018253             59
 13230  6018254             61
 
 [13231 rows x 2 columns],
         YKR_ID  car_r_t_Dixi
 0      5785640            45
 1      5785641            45
 2      5785642            52
 3      5785643            48
 4      5787544            44
 ...        ...           ...
 13226  6016698            63
 13227  6016699            65
 13228  6018252            -1
 13229  6018253            64
 13230  6018254            66
 
 [13231 rows x 2 columns],
         YKR_ID  car_r_t_Myyrmanni
 0      5785640                 39
 1      5785641                 39
 2      5785642                 47
 3      5785643                 51
 4      5787544                 38


In [50]:
# I'll do the merging manually as in ex 3 pr 2 since I haven't solved the problem with the for-loop

grid = grid.merge(filelist[0], on="YKR_ID", how="left")
grid = grid.merge(filelist[1], on="YKR_ID", how="left")
grid = grid.merge(filelist[2], on="YKR_ID", how="left")
grid = grid.merge(filelist[3], on="YKR_ID", how="left")
grid = grid.merge(filelist[4], on="YKR_ID", how="left")
grid = grid.merge(filelist[5], on="YKR_ID", how="left")
grid = grid.merge(filelist[6], on="YKR_ID", how="left")

In [51]:
# Check that there's a correct number of columns

assert len(grid.columns) >= 9, "There are some columns missing from the grid."

In [52]:
grid

,x,y,YKR_ID,geometry,car_r_t_Jumbo,car_r_t_Dixi,car_r_t_Myyrmanni,car_r_t_Itis,car_r_t_Forum,car_r_t_IsoOmena,car_r_t_Ruoholahti
0,381875.0,6697880.0,5785640,"POLYGON ((382000.000 6697750.000, 381750.000 6...",39,45,39,50,49,56,56
1,382125.0,6697880.0,5785641,"POLYGON ((382250.000 6697750.000, 382000.000 6...",39,45,39,51,49,56,56
2,382375.0,6697880.0,5785642,"POLYGON ((382500.000 6697750.000, 382250.000 6...",45,52,47,58,57,64,63
3,382625.0,6697880.0,5785643,"POLYGON ((382750.000 6697750.000, 382500.000 6...",46,48,51,57,60,67,67
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000 6697500.000, 381000.000 6...",38,44,38,50,48,55,55
...,...,...,...,...,...,...,...,...,...,...,...
13226,372875.0,6665630.0,6016698,"POLYGON ((373000.000 6665500.000, 372750.000 6...",58,63,54,60,48,31,44
13227,373125.0,6665630.0,6016699,"POLYGON ((373250.000 6665500.000, 373000.000 6...",60,65,56,62,50,33,46
13228,372375.0,6665380.0,6018252,"POLYGON ((372500.000 6665250.000, 372250.000 6...",-1,-1,-1,-1,-1,-1,-1
13229,372625.0,6665380.0,6018253,"POLYGON ((372750.000 6665250.000, 372500.000 6...",59,64,55,61,49,32,45


In [53]:
# Replace all no-data values with NaN and drop the rows containing NaN.

grid = grid.replace(to_replace=-1, value=np.nan)

grid = grid.dropna(axis=0, how='any')

len(grid)

13140

In [54]:
# Find out the shortest travel time to any shopping center and store it in the column "min_t"

grid['min_t_c'] = grid[['car_r_t_Jumbo', 'car_r_t_Dixi', 'car_r_t_Myyrmanni', 'car_r_t_Itis', 'car_r_t_Forum', 'car_r_t_IsoOmena', 'car_r_t_Ruoholahti']].min(axis=1)

# Find out the column name for the shortest travel time and store it in the column "dominant_service"

grid['dominant_service'] = grid[['car_r_t_Jumbo', 'car_r_t_Dixi', 'car_r_t_Myyrmanni', 'car_r_t_Itis', 'car_r_t_Forum', 'car_r_t_IsoOmena', 'car_r_t_Ruoholahti']].idxmin(axis=1)

grid["dominant_service"].replace({"car_r_t_Jumbo": "Jumbo", "car_r_t_Dixi": "Dixi", "car_r_t_Myyrmanni" : "Myyrmanni", "car_r_t_Itis": "Itis", "car_r_t_Forum": "Forum", "car_r_t_IsoOmena": "IsoOmena", "car_r_t_Ruoholahti": "Ruoholahti"}, inplace=True)

In [55]:
grid

,x,y,YKR_ID,geometry,car_r_t_Jumbo,car_r_t_Dixi,car_r_t_Myyrmanni,car_r_t_Itis,car_r_t_Forum,car_r_t_IsoOmena,car_r_t_Ruoholahti,min_t_c,dominant_service
0,381875.0,6697880.0,5785640,"POLYGON ((382000.000 6697750.000, 381750.000 6...",39.0,45.0,39.0,50.0,49.0,56.0,56.0,39.0,Jumbo
1,382125.0,6697880.0,5785641,"POLYGON ((382250.000 6697750.000, 382000.000 6...",39.0,45.0,39.0,51.0,49.0,56.0,56.0,39.0,Jumbo
2,382375.0,6697880.0,5785642,"POLYGON ((382500.000 6697750.000, 382250.000 6...",45.0,52.0,47.0,58.0,57.0,64.0,63.0,45.0,Jumbo
3,382625.0,6697880.0,5785643,"POLYGON ((382750.000 6697750.000, 382500.000 6...",46.0,48.0,51.0,57.0,60.0,67.0,67.0,46.0,Jumbo
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000 6697500.000, 381000.000 6...",38.0,44.0,38.0,50.0,48.0,55.0,55.0,38.0,Jumbo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13225,372625.0,6665630.0,6016697,"POLYGON ((372750.000 6665500.000, 372500.000 6...",57.0,62.0,53.0,59.0,46.0,30.0,43.0,30.0,IsoOmena
13226,372875.0,6665630.0,6016698,"POLYGON ((373000.000 6665500.000, 372750.000 6...",58.0,63.0,54.0,60.0,48.0,31.0,44.0,31.0,IsoOmena
13227,373125.0,6665630.0,6016699,"POLYGON ((373250.000 6665500.000, 373000.000 6...",60.0,65.0,56.0,62.0,50.0,33.0,46.0,33.0,IsoOmena
13229,372625.0,6665380.0,6018253,"POLYGON ((372750.000 6665250.000, 372500.000 6...",59.0,64.0,55.0,61.0,49.0,32.0,45.0,32.0,IsoOmena


In [58]:
# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
grid["geoid"] = grid.index.astype(str)

Okey, let's visualize the data. I'll make an interactive map showing the travel times to closest mall and also the places of different malls.

In [ ]:
# Create a Map instance

m = folium.Map(location=[60.25, 24.8], tiles = "cartodbpositron", zoom_start=10, control_scale=True)

# Plot a choropleth map

folium.Choropleth(
    geo_data=grid,
    name="Travel time to the closest malls by car, min",
    data=grid,
    columns=["geoid","min_t_c"],
    key_on='feature.id',
    fill_color="RdYlBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0).add_to(m)

#Show map
m